In [1]:
import pybamm as pb
import pybamm
import copy
import numpy as np

C:\Anaconda3\lib\site-packages\pybamm\expression_tree\functions.py:204: RuntimeWarning: invalid value encountered in sign
  return self.function(*evaluated_children)


In [2]:
# ambient and initial temperature in kelvin
TEMPERATURE = 300.0

# battery type can be cylindric or prismatic
BATTERY_TYPE = "cylindric"

# diameter in m of the cylindric cell. Ignored when another geometry is selected
DIAMETER = 0.02

# widths in m of the prismatic cell. Ignored when another geometry is selected
WIDTH_A = 0.02
WIDTH_B = 0.02

# height
HEIGHT = 0.05

In [3]:
# generator to split list into chunks
def chunks(lst, n):
    if not n:
        raise Exception("n must be a positive integer")

    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [4]:
def merge_parameter_values(p_A, p_B):
    
    dict_a = dict(p_A)
    dict_b = dict(p_B)
    
    merged_dict = copy.deepcopy(dict_a)
    
    for the_key, the_value in dict_b.items():
        
        if the_key not in merged_dict.keys():
            
            merged_dict.update({the_key: the_value})
    
    return pb.ParameterValues(merged_dict)

In [5]:
def calc_geometry(battery_type=BATTERY_TYPE, diameter=DIAMETER, width_a=WIDTH_A, width_b=WIDTH_B, height=HEIGHT):
    
    if battery_type == 'cylindric':
        
        area = diameter**2 * np.pi / 2 + diameter * np.pi * height
        
        volume = diameter**2 * np.pi * height / 4
    
    elif battery_type == 'prismatic':
        
        area = 2 * (width_a * width_b + width_a * height + width_b * height)
        
        volume = width_a * width_b * height
    
    else:
        
        raise Exception("Unsupported battery type: {0}".format(battery_type))
    
    return area, volume
    

In [6]:
area, volume = calc_geometry()

In [7]:
# lumped with arbitrary geometry specified 
options = {
    "cell geometry": "arbitrary",
    "thermal": "lumped",
    "SEI": "reaction limited",
    "particle": "Fickian diffusion",
    "particle mechanics": "swelling and cracking"
}

model = pybamm.lithium_ion.DFN(options)

In [8]:
output_variables = [
    "Terminal voltage [V]",
    'Negative particle surface tangential stress [Pa]',
    'Negative particle surface radial stress [Pa]',
    'X-averaged negative particle surface radial stress [Pa]',
    'X-averaged negative particle surface tangential stress [Pa]',
    'Positive particle surface tangential stress [Pa]',
    'Positive particle surface radial stress [Pa]',
    'X-averaged positive particle surface radial stress [Pa]',
    'X-averaged positive particle surface tangential stress [Pa]',
    'Negative particle surface displacement [m]',
    'Positive particle surface displacement [m]'
    ]

In [9]:
param_keys = ['Negative current collector thickness [m]',
 'Negative electrode thickness [m]',
 'Separator thickness [m]',
 'Positive electrode thickness [m]',
 'Positive current collector thickness [m]',
 'Electrode height [m]',
 'Electrode width [m]',
 'Cell cooling surface area [m2]',
 'Cell volume [m3]',
 'Negative current collector density [kg.m-3]',
 'Positive current collector density [kg.m-3]',
 'Negative current collector specific heat capacity [J.kg-1.K-1]',
 'Positive current collector specific heat capacity [J.kg-1.K-1]',
 'Negative current collector thermal conductivity [W.m-1.K-1]',
 'Positive current collector thermal conductivity [W.m-1.K-1]',
 'Nominal cell capacity [A.h]',
 'Typical current [A]',
 'Current function [A]',
 'Negative electrode conductivity [S.m-1]',
 'Maximum concentration in negative electrode [mol.m-3]',
 'Negative electrode diffusivity [m2.s-1]',
 'Negative electrode OCP [V]',
 'Negative electrode porosity',
 'Negative electrode active material volume fraction',
 'Negative particle radius [m]',
 'Negative electrode Bruggeman coefficient (electrolyte)',
 'Negative electrode electrons in reaction',
 'Negative electrode exchange-current density [A.m-2]',
 'Negative electrode density [kg.m-3]',
 'Negative electrode specific heat capacity [J.kg-1.K-1]',
 'Negative electrode thermal conductivity [W.m-1.K-1]',
 'Negative electrode OCP entropic change [V.K-1]',
 'Positive electrode conductivity [S.m-1]',
 'Maximum concentration in positive electrode [mol.m-3]',
 'Positive electrode diffusivity [m2.s-1]',
 'Positive electrode OCP [V]',
 'Positive electrode porosity',
 'Positive electrode active material volume fraction',
 'Positive particle radius [m]',
 'Positive electrode Bruggeman coefficient (electrolyte)',
 'Positive electrode electrons in reaction',
 'Positive electrode exchange-current density [A.m-2]',
 'Positive electrode density [kg.m-3]',
 'Positive electrode specific heat capacity [J.kg-1.K-1]',
 'Positive electrode thermal conductivity [W.m-1.K-1]',
 'Positive electrode OCP entropic change [V.K-1]',
 'Separator porosity',
 'Separator Bruggeman coefficient (electrolyte)',
 'Separator density [kg.m-3]',
 'Separator specific heat capacity [J.kg-1.K-1]',
 'Separator thermal conductivity [W.m-1.K-1]',
 'Typical electrolyte concentration [mol.m-3]',
 'Cation transference number',
 '1 + dlnf/dlnc',
 'Electrolyte diffusivity [m2.s-1]',
 'Electrolyte conductivity [S.m-1]',
 'Reference temperature [K]',
 'Ambient temperature [K]',
 'Total heat transfer coefficient [W.m-2.K-1]',
 'Number of electrodes connected in parallel to make a cell',
 'Number of cells connected in series to make a battery',
 'Lower voltage cut-off [V]',
 'Upper voltage cut-off [V]',
 'Initial concentration in negative electrode [mol.m-3]',
 'Initial concentration in positive electrode [mol.m-3]',
 'Initial concentration in electrolyte [mol.m-3]',
 'Initial temperature [K]'] + ['Inner SEI partial molar volume [m3.mol-1]',
 'Outer SEI partial molar volume [m3.mol-1]',
 'SEI reaction exchange current density [A.m-2]',
 'SEI resistivity [Ohm.m]',
 'Initial inner SEI thickness [m]',
 'Initial outer SEI thickness [m]']

In [10]:
parameter_values_dict = dict(pybamm.ParameterValues(chemistry=pybamm.parameter_sets.Marquis2019))

required_parameter_values_dict = {}

for elem in parameter_values_dict.keys():
    
    if elem in param_keys:
        
        required_parameter_values_dict.update({elem: parameter_values_dict[elem]})

# assert len(required_parameter_values_dict) == len(param_keys)

parameter_values = pb.ParameterValues(required_parameter_values_dict)

ai_params = pb.ParameterValues(chemistry=pb.parameter_sets.Ai2020)

additional_parameter_values_dict = {
    'Negative electrode cracking rate': ai_params['Negative electrode cracking rate'],
    'Negative electrode activation energy for cracking rate [J.mol-1]': ai_params['Negative electrode activation energy for cracking rate [J.mol-1]'],
    "Negative electrode Young's modulus [Pa]": ai_params["Negative electrode Young's modulus [Pa]"],
    "Negative electrode Paris' law constant m": ai_params["Negative electrode Paris' law constant m"],
    "Negative electrode initial crack length [m]": ai_params["Negative electrode initial crack length [m]"],
    "Negative electrode partial molar volume [m3.mol-1]": ai_params["Negative electrode partial molar volume [m3.mol-1]"],
    "Negative electrode Poisson's ratio": ai_params["Negative electrode Poisson's ratio"],
    "Negative electrode Paris' law constant b": ai_params["Negative electrode Paris' law constant b"],
    'Positive electrode cracking rate': ai_params['Positive electrode cracking rate'],
    'Positive electrode activation energy for cracking rate [J.mol-1]': ai_params['Positive electrode activation energy for cracking rate [J.mol-1]'],
    "Positive electrode Young's modulus [Pa]": ai_params["Positive electrode Young's modulus [Pa]"],
    "Positive electrode Paris' law constant m": ai_params["Positive electrode Paris' law constant m"],
    "Positive electrode initial crack length [m]": ai_params["Positive electrode initial crack length [m]"],
    "Positive electrode partial molar volume [m3.mol-1]": ai_params["Positive electrode partial molar volume [m3.mol-1]"],
    "Positive electrode Poisson's ratio": ai_params["Positive electrode Poisson's ratio"],
    "Positive electrode Paris' law constant b": ai_params["Positive electrode Paris' law constant b"],
    'Negative electrode reference concentration for free of deformation [mol.m-3]': ai_params['Negative electrode reference concentration for free of deformation [mol.m-3]'],
    'Positive electrode reference concentration for free of deformation [mol.m-3]': ai_params['Negative electrode reference concentration for free of deformation [mol.m-3]'],
    'Negative electrode number of cracks per unit area [m-2]': ai_params['Negative electrode number of cracks per unit area [m-2]'],
    'Positive electrode number of cracks per unit area [m-2]': ai_params['Positive electrode number of cracks per unit area [m-2]'],
    'Negative electrode initial crack width [m]': ai_params['Negative electrode initial crack width [m]'],
    'Positive electrode initial crack width [m]': ai_params['Positive electrode initial crack width [m]'],
    'Cell thermal expansion coefficient [m.K-1]': ai_params['Cell thermal expansion coefficient [m.K-1]'],
    'Negative electrode volume change': ai_params['Negative electrode volume change'],  # "[data]graphite_volume_change_Ai2020",
    'Positive electrode volume change': ai_params['Positive electrode volume change']
}

additional_parameter_values = pb.ParameterValues(additional_parameter_values_dict)

parameter_values = merge_parameter_values(additional_parameter_values, parameter_values)

parameter_values['Positive electrode diffusivity [m2.s-1]'] = "[ml data]lico2_diffusivity_Dualfoil1998"
# parameter_values['Positive electrode OCP [V]'] = "[ml data]lico2_diffusivity_Dualfoil1998"

In [11]:
parameter_values["Initial temperature [K]"] = TEMPERATURE
parameter_values["Ambient temperature [K]"] = TEMPERATURE
parameter_values["Cell cooling surface area [m2]"] = area
parameter_values["Cell volume [m3]"] = volume

In [12]:
experiment = pb.Experiment(
    [
        (
            "Charge at C/10 until 4.05 V",
            "Hold at 4.05 V until C/10",
            "Rest for 5 minutes",
            "Discharge at 1 C until 3.3 V",
            "Rest for 5 minutes",
        )
    ]
    * 2
    + [
        (
            "Charge at 1 C until 4.05 V",
            "Hold at 4.05 V until C/20",
            "Rest for 30 minutes",
            "Discharge at C/3 until 3.3 V",
            "Rest for 30 minutes",
        ),
        (
            "Charge at 1 C until 4.05 V",
            "Hold at 4.05 V until C/20",
            "Rest for 30 minutes",
            "Discharge at 1 C until 3.3 V",
            "Rest for 30 minutes",
        ),
        (
            "Charge at 1 C until 4.05 V",
            "Hold at 4.05 V until C/20",
            "Rest for 30 minutes",
            "Discharge at 2 C until 3.3 V",
            "Rest for 30 minutes",
        ),
        (
            "Charge at 1 C until 4.05 V",
            "Hold at 4.05 V until C/20",
            "Rest for 30 minutes",
            "Discharge at 3 C until 3.3 V (10 second period)",
            "Rest for 30 minutes",
        ),
    ]
)

In [13]:
# pick solver 
solver = pybamm.CasadiSolver(mode="safe")

sim = pb.Simulation(model, parameter_values=parameter_values, experiment=experiment, solver=solver)

sim.solve()

C:\Anaconda3\lib\site-packages\pybamm\expression_tree\functions.py:204: RuntimeWarning: invalid value encountered in sign
  return self.function(*evaluated_children)


In [15]:
for temp_vars in chunks(output_variables, 6):
    sim.plot(temp_vars, n_rows=3, figsize=(8, 12))

interactive(children=(FloatSlider(value=0.0, description='t', max=32.089795874522025, step=0.32089795874522026…

interactive(children=(FloatSlider(value=0.0, description='t', max=32.089795874522025, step=0.32089795874522026…